# Homework no: 7 - Mid Semester
### Name : ANGGARA PUTRA PRATAMA - 1101174240 
github.com/anggaraputrapratama

In [21]:
%matplotlib inline
import numpy as np
import pandas as pd
import pathlib
#import matplotlib.pyplot as plt
#import matplotlib.patches as mpatches
from skimage.util.shape import view_as_windows
from scipy.signal import find_peaks, peak_prominences
#from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier
from pathlib import Path 
import glob
import os
import re
import math

In [23]:
#sort dataset sesuai urutan nama file
path = r'data'
path1 = r'data/label'
data_files = pathlib.Path(path)
files = sorted(data_files.glob('*.csv'))
file_pattern = re.compile(r'.*?(\d+).*?')
def get_order(file):
    match = file_pattern.match(Path(file).name)
    if not match:
        return math.inf
    return int(match.groups()[0])

sorted_files = sorted(files, key=get_order)
print(sorted_files)

[PosixPath('data/1.csv'), PosixPath('data/2.csv'), PosixPath('data/3.csv'), PosixPath('data/4.csv'), PosixPath('data/5.csv'), PosixPath('data/6.csv'), PosixPath('data/7.csv'), PosixPath('data/8.csv'), PosixPath('data/9.csv'), PosixPath('data/10.csv'), PosixPath('data/11.csv'), PosixPath('data/12.csv'), PosixPath('data/13.csv'), PosixPath('data/14.csv'), PosixPath('data/15.csv')]


In [14]:
for k, f in enumerate(sorted_files,1):
    print(f)
    data = np.genfromtxt(fname=f, delimiter=',', usecols=(1,2,3,4), dtype='int32')
    mask = data[:,-1] > 0
    data1 = data[mask,]
    #print (data1)
    ax = data1[:,0]
    ay = data1[:,1]
    az = data1[:,2]
    am = np.sqrt(ax*ax+ay*ay+az*az)

    # Use these options determine the way floating point numbers
    np.set_printoptions(precision=3, suppress=True)

    # Convert am into two dimensional array with reshape to equal with x, y, z
    am = am.reshape(-1,1)
    #print(am.shape)

    # Use Stack arrays in sequence horizontally to concate am into data
    data2 = np.hstack([data1, am])
    # Show the data
    #print (data2)
    #print (len(data2), data2.shape)
    #Given id for each label
    axis_label = {0:'ax', 1:'ay', 2:'az', 4:'am'}
    #Frequency Sampling
    fs = 52
    
    #Use for loop to plot the figure of label 1
    for i in range (1,8):
        mask1 = data2[:,3]==i
        #Assign label
        label_list = []
        header = 'ax_b, ax_dc, ax_ac, ay_b, ay_dc, ay_ac, az_b, az_dc, az_ac, am_b, am_dc, am_ac'
        for j in axis_label:
            alx = data2[mask1,j]
        
            #Cut off 10% from two ends
            n = len(alx)
            #Use int to define cut off
            cut_len = int(n*0.1)
            b = alx[cut_len:-cut_len]
            m = b.size
        
            #Convert data into frequency domain
            freqY = np.fft.fft(b)
            freq = np.fft.fftfreq(m, d=1/fs)
            shift_freq = np.fft.fftshift(freq)
                
            #Spectrum
            spectrum = np.sqrt(freqY.real**2+freqY.imag**2)               
            shift_spec = np.fft.fftshift(spectrum)
            spec = np.fft.ifft(spectrum)                   
                
            #Low pass filter
            lowPassMask = abs(freq) <=1 #Cut off 1hz
            lowPassFy = freqY.copy()
            lowPassFy[~lowPassMask] = 0 # ~, equivalent to logical_not
            dc = np.fft.ifft(lowPassFy)
        
            #High pass filter
            highPassFy = freqY.copy()
            highPassFy[lowPassMask] = 0 
            ac = np.fft.ifft(highPassFy)
        
            #Reshape b, dc, ac and append to label list 
            label_list.append(b.reshape(-1,1))
            label_list.append(dc.real.reshape(-1,1))
            label_list.append(ac.real.reshape(-1,1))
            
            user_path = data_files.joinpath('user{}'.format(k))            
            if not user_path.exists():
                user_path.mkdir()
            path1= r'data/user{}'.format(k)
            label1_path= pathlib.Path(path1)
            label_path = label1_path.joinpath('label')            
            if not label_path.exists():
                label_path.mkdir()
            arrange_table = np.hstack(label_list)
            np.savetxt(r'data/user{}/label/label_{}.csv'.format(k,i), arrange_table, delimiter=',', fmt='%18f', header=header,comments='# ')
print('SUDAH TAHAP 1')

data/1.csv
data/2.csv
data/3.csv
data/4.csv
data/5.csv
data/6.csv
data/7.csv
data/8.csv
data/9.csv
data/10.csv
data/11.csv
data/12.csv
data/13.csv
data/14.csv
data/15.csv
SUDAH TAHAP 1


In [22]:
for h in range (1,16):
    path = r'data/user{}/label'.format(h)
    data_file = pathlib.Path(path)
    files = sorted(data_file.glob('label*.csv'))

    #Define function for mean minmax sum calculation
    def mean_minmax_sum(Nilai):
        minmax = np.zeros(Nilai.shape[0], dtype=np.float32)
        e=Nilai.flatten()
        for i in range(Nilai.shape[0]):
            min_peaks, _ = find_peaks(-e) 
            max_peaks, _ = find_peaks(e) 

            n = min(min_peaks.size, max_peaks.size)
            if n > 0:
                loc_min = e[min_peaks[:n]] 
                loc_max = e[max_peaks[:n]] 
                differences = loc_max - loc_min
                average = differences.sum() / n
            else:
                average = np.nan
            minmax[i] = average
        return minmax

    #Define function for rms velocity calculation
    def rms_velocity(x):
        z = view_as_windows(x, (10), 1)
        v = np.sum(z, axis = 1)
        z = view_as_windows(v, (52,), 26)
        rms = np.sqrt(np.mean(np.square(z), axis = 1))
        return rms
    #Calculation process
    for j, f in enumerate(files,0):
        print(h, f.name)
        #Use pandas dataframe to read dataset
        df = pd.read_csv(f)
        data = df.values
        feature_list = []
        header = ''
        for k in range(12):
            a = data[:,k]
            b = view_as_windows(a, (52,), 26)
            mean = np.mean(b, axis = 1)
            std = np.std(b, axis = 1)
            minmax = mean_minmax_sum(b)
            rms = rms_velocity(a)

            nrows = min(b.shape[0], rms.shape[0], minmax.shape[0])
            feature_list.append(mean[:nrows].reshape(-1,1))
            feature_list.append(std[:nrows].reshape(-1,1))
            feature_list.append(minmax[:nrows].reshape(-1,1))
            feature_list.append(rms[:nrows].reshape(-1,1))

            #Store the results
            header += 'mean_{0}, std_{0}, minmax_{0}, rms_{0},'.format(df.columns[k])
            path2= r'data/user{}'.format(h)
            features1_path= pathlib.Path(path2)
            features_path = features1_path.joinpath('features')            
            if not features_path.exists():
                features_path.mkdir()
            arrange = np.hstack(feature_list)
            print(arrange.shape)
            np.savetxt(r'data/user{}/features/features_{}.csv'.format(h,j), arrange, delimiter=',', fmt='%18f', header=header, comments='')

1 label_1.csv
(1034, 4)
(1034, 8)
(1034, 12)
(1034, 16)
(1034, 20)
(1034, 24)
(1034, 28)
(1034, 32)
(1034, 36)
(1034, 40)
(1034, 44)
(1034, 48)
1 label_2.csv
(27, 4)
(27, 8)
(27, 12)
(27, 16)
(27, 20)
(27, 24)
(27, 28)
(27, 32)
(27, 36)
(27, 40)
(27, 44)
(27, 48)
1 label_3.csv
(342, 4)
(342, 8)
(342, 12)
(342, 16)
(342, 20)
(342, 24)
(342, 28)
(342, 32)
(342, 36)
(342, 40)
(342, 44)
(342, 48)
1 label_4.csv
(825, 4)
(825, 8)
(825, 12)
(825, 16)
(825, 20)
(825, 24)
(825, 28)
(825, 32)
(825, 36)
(825, 40)
(825, 44)
(825, 48)
1 label_5.csv
(96, 4)
(96, 8)
(96, 12)
(96, 16)
(96, 20)
(96, 24)
(96, 28)
(96, 32)
(96, 36)
(96, 40)
(96, 44)
(96, 48)
1 label_6.csv
(88, 4)
(88, 8)
(88, 12)
(88, 16)
(88, 20)
(88, 24)
(88, 28)
(88, 32)
(88, 36)
(88, 40)
(88, 44)
(88, 48)
1 label_7.csv
(2575, 4)
(2575, 8)
(2575, 12)
(2575, 16)
(2575, 20)
(2575, 24)
(2575, 28)
(2575, 32)
(2575, 36)
(2575, 40)
(2575, 44)
(2575, 48)
2 label_1.csv
(1357, 4)
(1357, 8)
(1357, 12)
(1357, 16)
(1357, 20)
(1357, 24)
(1357, 28)

(2572, 4)
(2572, 8)
(2572, 12)
(2572, 16)
(2572, 20)
(2572, 24)
(2572, 28)
(2572, 32)
(2572, 36)
(2572, 40)
(2572, 44)
(2572, 48)
10 label_1.csv
(1354, 4)
(1354, 8)
(1354, 12)
(1354, 16)
(1354, 20)
(1354, 24)
(1354, 28)
(1354, 32)
(1354, 36)
(1354, 40)
(1354, 44)
(1354, 48)
10 label_2.csv
(104, 4)
(104, 8)
(104, 12)
(104, 16)
(104, 20)
(104, 24)
(104, 28)
(104, 32)
(104, 36)
(104, 40)
(104, 44)
(104, 48)
10 label_3.csv
(724, 4)
(724, 8)
(724, 12)
(724, 16)
(724, 20)
(724, 24)
(724, 28)
(724, 32)
(724, 36)
(724, 40)
(724, 44)
(724, 48)
10 label_4.csv
(680, 4)
(680, 8)
(680, 12)
(680, 16)
(680, 20)
(680, 24)
(680, 28)
(680, 32)
(680, 36)
(680, 40)
(680, 44)
(680, 48)
10 label_5.csv
(118, 4)
(118, 8)
(118, 12)
(118, 16)
(118, 20)
(118, 24)
(118, 28)
(118, 32)
(118, 36)
(118, 40)
(118, 44)
(118, 48)
10 label_6.csv
(227, 4)
(227, 8)
(227, 12)
(227, 16)
(227, 20)
(227, 24)
(227, 28)
(227, 32)
(227, 36)
(227, 40)
(227, 44)
(227, 48)
10 label_7.csv
(682, 4)
(682, 8)
(682, 12)
(682, 16)
(682, 2

In [24]:
#Read several dataset in one folder using pathlib and glob
path = r'data/user1/features'
data_path = pathlib.Path(path)
data_files = sorted(data_path.glob('features*.csv'))

#Choose 100 points of samples
n = 100
np.random.seed(100)
fea_lst = []
y_lst = []
for i, f in enumerate(data_files):
    print(i, f.name)
    df = pd.read_csv(f)
    df.fillna(method='ffill', inplace = True)
    df.fillna(method='bfill', inplace = True)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    data = df.values
    idx = np.random.choice(data.shape[0], min(n, data.shape[0]),replace=False)
    arr = data[idx,:]
    fea_lst.append(arr)
    y_lst.append(np.full(arr.shape[0], i))
    print('df.shape {}, select {} features'.format(df.shape, arr.shape[0]))
fea_name = list(df.columns)
x_data = np.vstack(fea_lst)
y_label = np.hstack(y_lst)
print(y_label)
print('x_data:',x_data.shape)
print('y_label:',y_label.shape)
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(x_data, y_label) # Build a forest of trees from the training set (X, y)
print("mean accuracy:",clf.score(x_data, y_label)) # Returns the mean accuracy
# print(clf.feature_importances_)
fea_dict= {}
for i, fea in enumerate(fea_name):
    fea_dict[fea] = clf.feature_importances_[i]
fea_import = [(v, k) for k, v in fea_dict.items() ]
fea_import = sorted(fea_import, reverse = True)
print(fea_import)

0 features_0.csv
df.shape (1034, 48), select 100 features
1 features_1.csv
df.shape (27, 48), select 27 features
2 features_2.csv
df.shape (342, 48), select 100 features
3 features_3.csv
df.shape (825, 48), select 100 features
4 features_4.csv
df.shape (96, 48), select 96 features
5 features_5.csv
df.shape (88, 48), select 88 features
6 features_6.csv
df.shape (2575, 48), select 100 features
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 

In [25]:
#Read several dataset in one folder using pathlib and glob
for g in range(2,16):
    path = r'data/user{}/features'.format(g)
    data_path = pathlib.Path(path)
    data_files = sorted(data_path.glob('features*.csv'))

    fea_lst = []
    y_lst = []
    for u, f in enumerate(data_files):
        df = pd.read_csv(f)
        df.fillna(method='ffill', inplace = True)
        df.fillna(method='bfill', inplace = True)
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        data = df.values
        idx = np.random.choice(data.shape[0],data.shape[0],replace=False)
        arr = data[idx,:]
        fea_lst.append(arr)
        y_lst.append(np.full(arr.shape[0], u))
    x_test = np.vstack(fea_lst)
    y_test = np.hstack(y_lst)
    print("predict, {}".format(path))
    print("mean accuracy:",clf.score(x_test, y_test)) # Returns the mean accuracy

predict, data/user2/features
mean accuracy: 0.491601608705938
predict, data/user3/features
mean accuracy: 0.461734693877551
predict, data/user4/features
mean accuracy: 0.47104350146784096
predict, data/user5/features
mean accuracy: 0.7553473212466898
predict, data/user6/features
mean accuracy: 0.480648899188876
predict, data/user7/features
mean accuracy: 0.021786927843294022
predict, data/user8/features
mean accuracy: 0.4436819687647894
predict, data/user9/features
mean accuracy: 0.732391563867887
predict, data/user10/features
mean accuracy: 0.5343275906402675
predict, data/user11/features
mean accuracy: 0.0
predict, data/user12/features
mean accuracy: 0.18060295790671219
predict, data/user13/features
mean accuracy: 0.0
predict, data/user14/features
mean accuracy: 0.7185393258426966
predict, data/user15/features
mean accuracy: 0.1629372833280807
